### SQL Queries with Python

#### Import Lib..

In [8]:
import pandas as pd
import numpy as np
import psycopg2

In [9]:
# Database Configuration---
config= {'database': 'ccdb', 'user': 'postgres', 'password': 'kshma'}

In [10]:
# Establish Connection
try:
     with psycopg2.connect(**config) as con:
            print('Connected to the PostgreSQL Server.')
except (psycopg2.DatabaseError, Exception) as error:
    print(error)

Connected to the PostgreSQL Server.


#### Function To Run the SQL Queries

In [11]:
# Creating cursor obj, execute, then Fetch Data
def execute_query(q1, commit=False):
    try:
        with con.cursor() as cur:
            cur.execute("BEGIN")
            cur.execute(q1)
            if commit:
                con.commit()
            else: 
                data =cur.fetchall()
                return data
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)
        

#### SQL Queries...

Data Overview

In [5]:
# --> To check the available tables in the database..
query = """
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        AND table_type = 'BASE TABLE'
    """

output = [i[0] for i in execute_query(query)]
print(output)

['cc_detail', 'cust_detail']


In [12]:
# --> Columns in cc_detail
query = """
        SELECT column_name
        FROM information_schema.columns
        WHERE table_schema = 'public'
        AND table_name = 'cc_detail';
    """
cc_detail_cols = [i[0] for i in execute_query(query)]
print(cc_detail_cols, '\n\nNo. of Cols: ',len(cc_detail_cols))

['client_num', 'card_category', 'annual_fees', 'activation_30_days', 'customer_acq_cost', 'week_start_date', 'week_num', 'qtr', 'current_year', 'credit_limit', 'total_revolving_bal', 'total_trans_amt', 'total_trans_ct', 'avg_utilization_ratio', 'use_chip', 'exp_type', 'interest_earned', 'delinquent_acc', 'weeknum2', 'revenue', 'profit'] 

No. of Cols:  21


In [7]:
# --> Columns in cust_detail
query = """
        SELECT column_name
        FROM information_schema.columns
        WHERE table_schema = 'public'
        AND table_name = 'cust_detail';
    """
cust_detail_cols = [i[0] for i in execute_query(query)]
print(cust_detail_cols, '\n\nNo. of Cols: ',len(cust_detail_cols))

['client_num', 'customer_age', 'gender', 'dependent_count', 'education_level', 'marital_status', 'state_cd', 'zipcode', 'car_owner', 'house_owner', 'personal_loan', 'contact', 'customer_job', 'income', 'cust_satisfaction_score', 'agegroup', 'incomegroup'] 

No. of Cols:  17


In [ ]:
# Check for duplicates and remove cust_detail
query = """
    CREATE TABLE cust_detail_new AS
    Select distinct * from 
    cust_detail;
    Drop table cust_detail;
    ALTER TABLE cust_detail_new RENAME TO cust_detail;
    
    """
execute_query(query, commit=True)

In [8]:
# first 5 Rows of cc_detail 
query = """
        SELECT * from cc_detail
        limit 5
    """
df =pd.DataFrame(execute_query(query), columns =cc_detail_cols)
df

,client_num,card_category,annual_fees,activation_30_days,customer_acq_cost,week_start_date,week_num,qtr,current_year,credit_limit,...,total_trans_amt,total_trans_ct,avg_utilization_ratio,use_chip,exp_type,interest_earned,delinquent_acc,weeknum2,revenue,profit
0,708117933,Blue,360,0,70,2023-01-01,Week-1,Q1,2023,1880.00,...,2469,34,0.000,Chip,Bills,419.730,1,1,3249,710
1,708119658,Blue,340,0,44,2023-01-01,Week-1,Q1,2023,12836.00,...,2519,53,0.081,Swipe,Bills,554.180,0,1,3413,850
2,708121908,Blue,405,0,71,2023-01-01,Week-1,Q1,2023,22917.00,...,2045,45,0.000,Swipe,Grocery,265.850,0,1,2716,600
3,708123033,Silver,355,0,78,2023-01-01,Week-1,Q1,2023,11463.00,...,14511,105,0.000,Swipe,Fuel,3047.310,0,1,17913,3324
4,708125733,Blue,220,0,69,2023-01-01,Week-1,Q1,2023,1438.30,...,4311,77,0.000,Swipe,Entertainment,1207.080,0,1,5738,1358


In [9]:
# first 5 Rows of cust_detail 
query = """
        SELECT * from cust_detail
        limit 5
    """
df =pd.DataFrame(execute_query(query), columns =cust_detail_cols)
df

,client_num,customer_age,gender,dependent_count,education_level,marital_status,state_cd,zipcode,car_owner,house_owner,personal_loan,contact,customer_job,income,cust_satisfaction_score,agegroup,incomegroup
0,713126733,51,F,1,High School,Married,CA,91750,no,no,no,cellular,Selfemployeed,10214,1,50-60,Low
1,769576608,51,F,3,Graduate,Single,TX,91750,yes,yes,no,unknown,Retirees,38886,4,50-60,Med
2,709110633,48,F,4,Unknown,Married,FL,91750,yes,no,no,cellular,Blue-collar,11997,1,40-50,Low
3,754851783,43,M,4,Unknown,Single,IL,91750,no,no,yes,cellular,White-collar,61815,3,40-50,Med
4,720409608,39,F,3,Graduate,Married,FL,91750,yes,yes,no,unknown,Selfemployeed,14975,4,30-40,Low


#### Adding New Columns -cc_detail

In [21]:
# Creating and Adding data to Revenue in cc_details table
query ="""
    alter table cc_detail add column Revenue Numeric;
    update cc_detail set Revenue = Round((annual_fees+ 
    total_trans_amt + interest_earned), 0)
    """
execute_query(query, commit=True)

In [19]:
# Creating and Adding data to WeekNum2 in cc_details table
query ="""
    alter table cc_detail add column WeekNum2 Numeric;
    update cc_detail set 
    WeekNum2 = FLOOR((EXTRACT(DOY FROM week_start_date) - 
            EXTRACT(DOY FROM DATE '2024-01-01')) / 7) + 1
    """
execute_query(query, commit=True)

In [22]:
# Creating and Adding data to Profit in cc_details table
query ="""
    alter table cc_detail add column Profit Numeric;
    update cc_detail set 
    Profit = Round(revenue-(customer_acq_cost + total_trans_amt),0 )
    """
execute_query(query, commit=True)

#### Adding New Columns - cust_detail

In [112]:
# Creating and Adding data to AgeGroup  in cust_details table
query ="""
    alter table cust_detail add column AgeGroup Varchar;
    update cust_detail set 
    AgeGroup = case
        when customer_age<30 then '20-30'
        when customer_age < 40 then '30-40'
        when customer_age < 50 then '40-50'
        when customer_age < 60 then '50-60'
        when customer_age >= 60 then '60+'
        else 'Unknown' end
    """
execute_query(query, commit=True)

In [113]:
# Creating and Adding data to IncomeGroup in cust_details table
query ="""
    alter table cust_detail add column IncomeGroup Varchar;
    update cust_detail set 
    IncomeGroup = case
        when income<10000 then 'Very Low'
        when income < 35000 then 'Low'
        when income < 70000 then 'Med'
        when income >=70000 then 'High'
        else 'Unknown' end
    """
execute_query(query, commit=True)

## KPI

#### --> KPIs Calculation in cc_detail

In [10]:
# Total Revenue, Trans Count, Trans. Amount, Interest,
# annual_fees, customer_acq_cost and Profit
query= """
    Select sum(revenue) as Revenue,
    sum(total_trans_ct) as trans_ct,
    sum(total_trans_amt) as trans_amt,
    sum(interest_earned) as Interest,
    sum(annual_fees) as annual_fees,
    sum(customer_acq_cost) as customer_acq_cost,
    sum(profit) as profit
    from cc_detail;
    """
output =execute_query(query)
print(f'Total Profit: {output[0][0]}')
print(f'Total trans_ct: {output[0][1]}')
print(f'Total trans_amt: {output[0][2]}')
print(f'Total Interest: {output[0][3]}')
print(f'Total Annual Fees: {output[0][4]}')
print(f'Total Cust Acq Cost: {output[0][5]}')
print(f'Total Profit: {output[0][6]}')    

Total Profit: 110631090
Total trans_ct: 1311302
Total trans_amt: 89044026
Total Interest: 15686764.460
Total Annual Fees: 5900030
Total Cust Acq Cost: 1945872
Total Profit: 19641192


#### --> KPIs Calculation in cust_detail

In [11]:
# Avg cust_satisfaction_score, credit_limit, customer_age,
# No of client_num, and income
query= """
    Select 
    round(avg(cust_satisfaction_score), 2) as avg_CSS,
    round(avg(credit_limit), 2) as avg_credit_limit,
    round(avg(customer_age), 2) as avg_customer_age,
    round(avg(income), 2) as avg_income,
    count(distinct cust.client_num) as cust_count
     from cust_detail as cust
     join cc_detail as cc
     on cust.client_num= cc.client_num;
    """
output =execute_query(query)
print(f'Avg. Cust Statisfaction Score : {output[0][0]}')
print(f'Avg. Credit Limit: {output[0][1]}')
print(f'Avg. Customer Age: {output[0][2]}')
print(f'Avg. Income: {output[0][3]}')
print(f'Total Customers: {output[0][4]}')

Avg. Cust Statisfaction Score : 3.19
Avg. Credit Limit: 8635.64
Avg. Customer Age: 46.27
Avg. Income: 56976.10
Total Customers: 10108


## Transaction Report

## Report 1

#### --> Week on Week Revenue percent 

In [22]:
# CW, PW and WOW% caluclations
query= """
    with cte as
    (   Select weeknum2, 
        sum(revenue) as current_week_revenue,
        lag(sum(revenue)) over(order by weeknum2) as prev_week_revenue
        from cc_detail
        group by weeknum2
    )
    select weeknum2, current_week_revenue, prev_week_revenue, 
    round(((current_week_revenue-prev_week_revenue)/prev_week_revenue)*100, 2
        ) as WOW_Revenue_percent
    from cte
    order by weeknum2 desc
    """
wow_rev= pd.DataFrame(execute_query(query),
        columns =['weeknum', 'current_week_revenue', 'prev_week_revenue', 'WOW_Revenue%'])
wow_rev.head()

,weeknum,current_week_revenue,prev_week_revenue,WOW_Revenue%
0,52,1866274,2140888,-12.83
1,51,2140888,2053106,4.28
2,50,2053106,1960320,4.73
3,49,1960320,2017560,-2.84
4,48,2017560,2094232,-3.66


#### --> Revenue By Card Category

In [12]:
# Sum of revenue by each card category
query= """
    Select 
    card_category, 
    sum(revenue) as revenue
    from cc_detail
    group by card_category
    order by revenue desc
    """
rev_by_cardCat= pd.DataFrame(execute_query(query),
            columns =['card_category', 'revenue'])
rev_by_cardCat

,card_category,revenue
0,Blue,92279042
1,Silver,11172686
2,Gold,4908146
3,Platinum,2271216


#### --> Revenue and Trans Count by qtr

In [13]:
# Sum of revenue and trans Count by each qtr
query= """
    Select 
    qtr, 
    sum(revenue) as revenue,
    sum(total_trans_ct) as total_trans_ct
    from cc_detail
    group by qtr
    order by qtr
    """
rev_And_trn_ct_by_qtr= pd.DataFrame(execute_query(query),
            columns =['qtr', 'revenue', 'total_trans_ct'])
rev_And_trn_ct_by_qtr

,qtr,revenue,total_trans_ct
0,Q1,27928802,326510
1,Q2,27641222,328402
2,Q3,28470962,333132
3,Q4,26590104,323258


#### --> Revenue and by Exp Type

In [14]:
# Sum of revenue by each exp type
query= """
    Select 
    exp_type, 
    sum(revenue) as revenue
    from cc_detail
    group by exp_type
    order by revenue desc 
    """
rev_by_exptype= pd.DataFrame(execute_query(query),
            columns =['exp_type', 'revenue'])
rev_by_exptype

,exp_type,revenue
0,Bills,27550294
1,Entertainment,19042340
2,Fuel,18654394
3,Grocery,17151656
4,Food,16500910
5,Travel,11731496


#### --> Revenue and by Customer Job

In [19]:
# Sum of revenue by each Customer Job
query= """
    Select 
    customer_job, 
    sum(revenue) as revenue
    from cc_detail as cc
    join cust_detail as cust
    on cc.client_num = cust.client_num
    group by customer_job
    order by revenue desc 
    """
rev_by_custJob= pd.DataFrame(execute_query(query),
            columns =['customer_job', 'revenue'])
rev_by_custJob

,customer_job,revenue
0,Businessman,34775700
1,White-collar,20229370
2,Selfemployeed,16523562
3,Govt,16223454
4,Blue-collar,13808614
5,Retirees,9070390


#### --> Revenue and by Income Group

In [20]:
# Sum of revenue by each exp type
query= """
    Select 
    IncomeGroup, 
    sum(cc.revenue) as revenue
    from cust_detail as cust
    join cc_detail as cc
    on cust.client_num = cc.client_num
    group by IncomeGroup
    order by revenue desc 
    """
rev_by_custJob= pd.DataFrame(execute_query(query),
            columns =['IncomeGroup', 'revenue'])
rev_by_custJob

,IncomeGroup,revenue
0,High,58459936
1,Med,31708536
2,Low,19421914
3,Very Low,1040704


## Report 2

#### --> Percent of Total By Card Category

In [13]:
# percent total by card category
query= """
    with cte as (
        Select sum(revenue) as total_revenue
        from cc_detail
    )
    select card_category, 
        sum(revenue) as revenue, 
        (select * from cte) as total_revenue,
        round(sum(revenue)*100/(select * from cte), 2) as "%_total"
    from cc_detail
    group by card_category
    order by "%_total" desc
    """
per_total= pd.DataFrame(execute_query(query),
        columns =['card_category', 'revenue','total_revenue', '%_total'])
per_total

,card_category,revenue,total_revenue,%_total
0,Blue,92279042,110631090,83.41
1,Silver,11172686,110631090,10.10
2,Gold,4908146,110631090,4.44
3,Platinum,2271216,110631090,2.05


#### --> Tran. Count by Week and Gender

In [42]:
# Total Transaction count by Week and Gender
query= """
    Select 
    week_start_date, 
    Gender,
    count(cc.total_trans_ct) as total_trans_ct
    from cust_detail as cust
    join cc_detail as cc
    on cust.client_num = cc.client_num
    group by week_start_date ,gender
	order by week_start_date
    """
trn_by_week_gender= pd.DataFrame(execute_query(query),
            columns =['week_start_date', 'Gender','total_trans_ct'])
trn_by_week_gender.head()

,week_start_date,Gender,total_trans_ct
0,2023-01-01,F,250
1,2023-01-01,M,138
2,2023-01-08,F,248
3,2023-01-08,M,142
4,2023-01-15,M,166


#### --> Total Annual Fees by Card Category

In [52]:
# Annual Fees by Card Category
query= """
    Select 
    card_category, 
    sum(annual_fees) as annual_fees
    from cc_detail
    group by card_category
    order by annual_fees desc
    """
afee_by_card_cat= pd.DataFrame(execute_query(query),
            columns =['card_category', 'annual_fees'])
afee_by_card_cat

,card_category,annual_fees
0,Blue,5371270
1,Silver,375010
2,Gold,112420
3,Platinum,41330


#### --> Revenue by merital Status and Gender

In [6]:
# Revenue by Merital Status and Gen
query= """
    Select 
    marital_status, 
    gender,
    sum(cc.revenue) as revenue
    from cust_detail as cust
    join cc_detail as cc
    on cust.client_num = cc.client_num
    group by marital_status ,gender
	order by marital_status
    """
rev_by_marsts_gender= pd.DataFrame(execute_query(query),
            columns =['marital_status', 'Gender','revenue'])
rev_by_marsts_gender.head()

,marital_status,Gender,revenue
0,Married,M,30830270
1,Married,F,25106078
2,Single,F,21611530
3,Single,M,24912688
4,Unknown,M,4700458


#### --> Revenue by Use Chip

In [56]:
# revenue by use chip
query= """
    Select 
    use_chip, 
    sum(revenue) as revenue
    from cc_detail
    group by use_chip
    order by revenue desc
    """
rev_by_usechip= pd.DataFrame(execute_query(query),
            columns =['use_chip', 'revenue'])
rev_by_usechip

,use_chip,revenue
0,Swipe,69825796
1,Chip,33933668
2,Online,6871626


#### --> Revenue and tran count Age Group

In [58]:
# Revenue and tran Age Group
query= """
    Select 
    AgeGroup, 
    sum(cc.revenue) as revenue,
    sum(cc.total_trans_ct) as total_trans_ct
    from cust_detail as cust
    join cc_detail as cc
    on cust.client_num = cc.client_num
    group by AgeGroup
	order by revenue desc
    """
rev_and_tran_by_agegrp= pd.DataFrame(execute_query(query),
            columns =['AgeGroup', 'revenue','total_trans_ct'])
rev_and_tran_by_agegrp

,AgeGroup,revenue,total_trans_ct
0,40-50,48567100,587488
1,50-60,36377624,399476
2,30-40,19163840,235736
3,60+,4417904,62666
4,20-30,2104622,25936


## Customer Report

## Report 3

#### --> Total Transaction amount by job and chip

In [7]:
# tran amt by customer job and use chip
query= """
    Select 
    customer_job, 
    use_chip,
    sum(cc.total_trans_amt) as total_trans_amt
    from cust_detail as cust
    join cc_detail as cc
    on cust.client_num = cc.client_num
    group by customer_job, use_chip
	order by customer_job asc,total_trans_amt desc
    """
tran_by_custjob_usechip= pd.DataFrame(execute_query(query),
            columns =['customer_job', 'use_chip','total_trans_ct'])
tran_by_custjob_usechip.head()

,customer_job,use_chip,total_trans_ct
0,Blue-collar,Swipe,7616446
1,Blue-collar,Chip,2722054
2,Blue-collar,Online,639176
3,Businessman,Swipe,14241178
4,Businessman,Chip,12293152


#### --> No. of clinets by income group and delinquent_acc status

In [40]:
# num of client by income grp and account status
query= """
    Select 
    IncomeGroup, 
    delinquent_acc,
    count(distinct cc.client_num) as client_num_count
    from cust_detail as cust
    join cc_detail as cc
    on cust.client_num = cc.client_num
    group by IncomeGroup, delinquent_acc
	
    """
num_clnts_by_incmgrp_dlqnt_ac= pd.DataFrame(execute_query(query),
            columns =['IncomeGroup', 'delinquent_acc','client_num_count'])
num_clnts_by_incmgrp_dlqnt_ac.head(6)

,IncomeGroup,delinquent_acc,client_num_count
0,High,0,2780
1,High,1,190
2,Low,0,3410
3,Low,1,219
4,Med,0,2974
5,Med,1,176


#### --> Avg. Cust Satisfaction Score by income Grp

In [45]:
# Avg CSS by income grp
query= """
    Select 
    IncomeGroup, 
    avg(cust_satisfaction_score) as css
    from cust_detail
    group by IncomeGroup
    order by css desc
	
    """
css_by_incmgrp= pd.DataFrame(execute_query(query),
            columns =['IncomeGroup', 'css'])
css_by_incmgrp.head(6)

,IncomeGroup,css
0,Very Low,3.2144846796657382
1,Med,3.1965079365079365
2,Low,3.1928906034720309
3,High,3.1740740740740741


#### --> Avg. Credit limit by income grp

In [49]:
# Avg. Credit Limit by income grp
query= """
    Select 
    IncomeGroup,
    avg(cc.credit_limit) as credit_limit
    from cust_detail as cust
    join cc_detail as cc
    on cust.client_num = cc.client_num
    group by IncomeGroup
    order by credit_limit desc
	
    """
avg_credit_limit_by_incmgrp= pd.DataFrame(execute_query(query),
            columns =['IncomeGroup', 'credit_limit'])
avg_credit_limit_by_incmgrp

,IncomeGroup,credit_limit
0,High,9745.6584175084175084
1,Very Low,9568.2498607242339833
2,Low,8302.0242491044364839
3,Med,7867.1188253968253968


#### --> Avg. Credit limit by Age Grp

In [50]:
# Avg. Credit Limit by Age grp
query= """
    Select 
    AgeGroup,
    avg(cc.credit_limit) as credit_limit
    from cust_detail as cust
    join cc_detail as cc
    on cust.client_num = cc.client_num
    group by AgeGroup
    order by credit_limit desc
	
    """
avg_credit_limit_by_agegrp= pd.DataFrame(execute_query(query),
            columns =['AgeGroup', 'credit_limit'])
avg_credit_limit_by_agegrp

,AgeGroup,credit_limit
0,40-50,8680.2187017001545595
1,50-60,8639.3596391580354160
2,30-40,8598.5288200108754758
3,20-30,8560.3116279069767442
4,60+,8393.9889097744360902


#### --> Avg Income by Age Grp

In [54]:
# Avg. Credit Limit by Age grp
query= """
    Select 
    AgeGroup,
    avg(income) as income
    from cust_detail
    group by AgeGroup
    order by income desc
	
    """
avg_income_by_agegrp= pd.DataFrame(execute_query(query),
            columns =['AgeGroup', 'income'])
avg_income_by_agegrp

,AgeGroup,income
0,50-60,63422.867023053792
1,40-50,56243.112166041069
2,30-40,54019.847743338771
3,20-30,47568.465116279070
4,60+,40968.088345864662


## Report 4

#### -->No client and percent by deliquent acc status

In [60]:
# No client by deliquent acc status and percent
query= """
    with cte as (
    select count(distinct client_num) as total_cust
    from cc_detail
    )
    
    Select 
      delinquent_acc,
      count( distinct client_num) as no_cust,
      (count( distinct client_num)*100/(select * from cte))
      as per_total
    from cc_detail
    group by delinquent_acc
    order by no_cust desc
    """
client_by_dlqntac= pd.DataFrame(execute_query(query),
            columns =['delinquent_acc', 'no_cust', 'per_total'])
client_by_dlqntac

,delinquent_acc,no_cust,per_total
0,0,9494,93
1,1,614,6


#### --> Trans count by Age Grp and house owners

In [69]:
# tran count by age and house owner
query= """
    Select 
    AgeGroup,
    house_owner,
    sum(cc.total_trans_ct) as total_trans_ct
    from cust_detail as cust
    join cc_detail as cc
    on cust.client_num = cc.client_num
    group by AgeGroup, house_owner
    order by total_trans_ct desc
    """
tran_cnt_by_agegrp_hsownr= pd.DataFrame(execute_query(query),
            columns =['AgeGroup', 'house_owner','credit_limit'])
tran_cnt_by_agegrp_hsownr.head()

,AgeGroup,house_owner,credit_limit
0,40-50,no,309780
1,40-50,yes,277708
2,50-60,no,209586
3,50-60,yes,189890
4,30-40,no,132224


#### --> Cust Satisfaction Score by income grp

In [16]:
# css by income grp
query= """
    Select 
    AgeGroup,
    avg(cust_satisfaction_score) as css
    from cust_detail
    group by AgeGroup
    order by css desc
    """
css_by_incomegrp= pd.DataFrame(execute_query(query),
            columns =['AgeGroup', 'css'])
css_by_incomegrp

,AgeGroup,css
0,60+,3.2669172932330827
1,30-40,3.2060902664491572
2,20-30,3.1953488372093023
3,50-60,3.1941196124290010
4,40-50,3.1697946566570987


#### --> Trans count By agegrp and personal load

In [16]:
# tran count by age and personal loan
query= """
    Select 
    AgeGroup,
    personal_loan,
    sum(cc.total_trans_ct) as total_trans_ct
    from cust_detail as cust
    join cc_detail as cc
    on cust.client_num = cc.client_num
    group by AgeGroup, personal_loan
    order by total_trans_ct desc
    """
tran_cnt_by_agegrp_prloan= pd.DataFrame(execute_query(query),
            columns =['AgeGroup', 'personal_loan','total_trans_ct'])
tran_cnt_by_agegrp_prloan.head()

,AgeGroup,personal_loan,total_trans_ct
0,40-50,no,512912
1,50-60,no,345844
2,30-40,no,209064
3,40-50,yes,74576
4,60+,no,54818


#### --> Trans count by age grp and car owner

In [15]:
# tran count by age and car owner
query= """
    Select 
    AgeGroup,
    car_owner,
    sum(cc.total_trans_ct) as total_trans_ct
    from cust_detail as cust
    join cc_detail as cc
    on cust.client_num = cc.client_num
    group by AgeGroup, car_owner
    order by total_trans_ct desc
    """
tran_cnt_by_agegrp_carownr= pd.DataFrame(execute_query(query),
            columns =['AgeGroup', 'car_owner','total_trans_ct'])
tran_cnt_by_agegrp_carownr.head()

,AgeGroup,car_owner,total_trans_ct
0,40-50,no,349490
1,50-60,no,246706
2,40-50,yes,237998
3,50-60,yes,152770
4,30-40,no,137140
